In [2]:
with open('data/input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print('length of the dataset in characters:', len(text))

length of the dataset in characters: 1115394


In [3]:
# Let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [4]:
# here are all the unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size) 


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s] # encoder: takes a string, outputs a list of integers
decode = lambda l: ''.join(itos[ix] for ix in l) # decoder: takes a list of integers, outputs a string

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


what is the difference between the two below code?

```decode = lambda l: ''.join([itos[ix] for ix in l])```

```decode = lambda l: ''.join(itos[ix] for ix in l)```


Let me explain the difference between these two code snippets. 

While they achieve the same result, there's a subtle syntactic difference:

decode = lambda l: ''.join([itos[ix] for ix in l])

This version uses a list comprehension [itos[ix] for ix in l]
It creates a full list in memory first, then joins it
The square brackets [] indicate that a new list object is being created

decode = lambda l: ''.join(itos[ix] for ix in l)

This version uses a generator expression (itos[ix] for ix in l)
It generates values one at a time without creating a full list
No square brackets means it's a generator expression
The main practical differences are:
Memory efficiency: The generator version (second one) is more memory efficient because it doesn't create an intermediate list. It generates each value on-the-fly as needed by join().

Performance:
For small sequences, the list comprehension might be slightly faster since it has less overhead

For large sequences, the generator version is better as it uses less memory
In most cases, the difference in performance would be negligible, but if you're working with very large sequences, the generator version (second one) would be the better choice due to its memory efficiency. Both will produce exactly the same output - they just differ in how they get there.

In [6]:
# tokenize the enitre dataset and store it in a tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
# data = data.unsqueeze(0)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# split the dataset into train and validation sets (90%)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [8]:
# Time Dimention of the input tensors feeding into the transformer

block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context}, the target: {target}")

when input is tensor([18]), the target: 47
when input is tensor([18, 47]), the target: 56
when input is tensor([18, 47, 56]), the target: 57
when input is tensor([18, 47, 56, 57]), the target: 58
when input is tensor([18, 47, 56, 57, 58]), the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target: 58


In [9]:
# Batch Dimention
# We are going to have mini batches of multiple chunks of text all stacked up in a single tensor
# We want to process multiple chunks at the same time since we are using GPUs which are good at parallel processing

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and taregs y
    data = train_data if split == 'train' else val_data
    # The reason we subtract block_size from len(data) in torch.randint(len(data) - block_size, (batch_size,)) is to ensure we can always get a complete sequence of length block_size starting from any randomly chosen index.
    ix = torch.randint(len(data)- block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)
print('targets: ')
print(yb.shape)
print(yb)

print('-----')
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t+1]
        target = yb[b, t]
        print("When input is ", context.tolist(), "the target: ", target.tolist())

inputs: 
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets: 
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----
When input is  [24] the target:  43
When input is  [24, 43] the target:  58
When input is  [24, 43, 58] the target:  5
When input is  [24, 43, 58, 5] the target:  57
When input is  [24, 43, 58, 5, 57] the target:  1
When input is  [24, 43, 58, 5, 57, 1] the target:  46
When input is  [24, 43, 58, 5, 57, 1, 46] the target:  43
When input is  [24, 43, 58, 5, 57, 1, 46, 43] the target:  39
When input is  [44] the target:  53
When input is  [44, 53] the target:  56
When input is  [44, 53, 56] the target:  1
When input is  [44, 53, 56, 1] the target:  58
When input is  [44, 53, 56, 1, 58] the targ

In [10]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch, Time, Channel)
        
        if targets is None:
            loss = None
        else:
            # pytorch cross entropy expects inputs of shape (B, C, T) so we need to reshape logits
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T) # this is equivalent to targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions 
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C) # we must pluck out the last element in the time dimention
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sample index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


m = BigramLanguageModel(vocab_size)
print(yb.shape)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss.item()) # the expected loss is -ln(1/65)
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))
# the generated text is garbage since it is in totally random level

torch.Size([4, 8])
torch.Size([32, 65])
4.878634929656982

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [12]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_size = 32

for steps in range(10000):
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())



2.382369041442871


In [13]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist()))
# this is a fairly simple model since the tokens are not talking to each other.
# Given any context we only llok at the very laast character to make the prediction for the next character.
# So, the tokens must start talking to each other, and they must know what is included in the context so that they can make better predictions about what will come next.




lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecor


#### The mathematical trick in self attention

In [14]:
# consider the following toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2  # batch, time, channel
x = torch.randn(B, T, C)
print(x.shape)
x.mean(dim=-1)

# What we try to achieve is that make the tokens at each row of the batch to talk to each other.
# But not in a way that for example the token at the fifth location talks to 6th, 7th, ... locations since those are future tokens in the sequence
# It should only talk to the 4th, 3rd, 2nd and 1st tokens
# So, the information only flows from previous context to the current time step, we can't get any information from future because we try to predict the future
# One way for the tokens to talk to each other is to take the avergae of the previous token as the feature vector for the t-th token

# We want x[b, t] = mean_{i<=t} x[b, i]
# version 1
xbow = torch.zeros((B, T, C)) # bow stands for Bag of Words
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #(t, C)
        xbow[b, t] = torch.mean(xprev, 0)

torch.Size([4, 8, 2])


In [15]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [16]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [17]:
# This current version is really inefficient since we are using two nested for loops
# We can use matrix multiplication 

# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T, T) @ (B, T, C) ----> (B, T, T) @ (B, T, C) this is a batch multiplication the result is (B, T, C)
torch.allclose(xbow, xbow2)

True

In [20]:
# version 3 

# this is an implementation of a simple average of the past tokens and the current token
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T)) # this is uniform, so each token is interesting in the same way
wei = wei.masked_fill(tril == 0, float('-inf')) # the future can not communicate with past
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow3, xbow)

RuntimeError: The size of tensor a (32) must match the size of tensor b (2) at non-singleton dimension 2

You can do weighted aggregation of the past elements by using matrix multiplication. The elements in the tril tells you how much of each element gives us into this position

In [28]:
# version 4: self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# the self-attention solves the problem of data dependency wehn gathering information from the past
# each token will emit two vectors: query and key
# query: what am I looking for?
# key: what do I contain?
# we do a dot product between queries and the keys, and this dot product becomes weights!

# this is a single head perform of Self Attention
head_size =16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)
# until now, no communication has happened yet since for each of the (B, T) we are producing a vector of keys and queries in parallel
wei = q @ k.transpose(-2, -1) # (B, T ,16) @ (B, 16, T) ---> (B, T, T)
tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ x
out.shape


torch.Size([4, 8, 32])

In [24]:
wei # every single batch has a different sort of weights
# when query and key find each other, and they will have a high affinity

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [27]:
# without masking and softmax
# version 4: self-attention

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)


head_size =16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, head_size)
q = query(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T ,16) @ (B, 16, T) ---> (B, T, T)
tril = torch.tril(torch.ones(T, T))
# wei = wei.masked_fill(tril == 0, float('-inf'))
# wei = F.softmax(wei, dim=-1)
out = wei @ x
out.shape
wei # how much information we need to aggregate from all of the tokens in the past

tensor([[[-1.7629e+00, -1.3011e+00,  5.6516e-01,  2.1616e+00, -1.0674e+00,
           1.9632e+00,  1.0765e+00, -4.5295e-01],
         [-3.3334e+00, -1.6556e+00,  1.0405e-01,  3.3782e+00, -2.1825e+00,
           1.0415e+00, -5.5714e-02,  2.9273e-01],
         [-1.0226e+00, -1.2606e+00,  7.6228e-02, -3.8125e-01, -9.8430e-01,
          -1.4303e+00,  7.4921e-02, -9.5465e-01],
         [ 7.8359e-01, -8.0143e-01, -3.3680e-01, -8.4963e-01, -5.6023e-01,
          -1.1701e+00, -1.2927e+00, -1.0260e+00],
         [-1.2566e+00,  1.8719e-02, -7.8797e-01, -1.3204e+00,  2.0363e+00,
           8.6381e-01,  3.7188e-01,  9.2577e-01],
         [-3.1262e-01,  2.4152e+00, -1.1059e-01, -9.9305e-01,  3.3449e+00,
          -2.5229e+00,  1.4187e+00,  1.2196e+00],
         [ 1.0876e+00,  1.9652e+00, -2.6213e-01, -3.1579e-01,  6.0905e-01,
           1.2616e+00, -5.4841e-01,  8.0485e-01],
         [-1.8044e+00, -4.1260e-01, -8.3061e-01,  5.8985e-01, -7.9869e-01,
          -5.8560e-01,  6.4332e-01,  6.3028e-01]],

In [30]:
value = nn.Linear(C, head_size, bias=False)
v = value(x) # (B, T, head_size)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

- Attnetion is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weight.

- There is no notion of space Attnetion simply acts as a set of vectors. This is why we need to positionally encode the tokens.

- Each example across each batch dimention is of course processed independently and never talk to each other.

- In an encoder attention block just delete the single line that does masking with ```tril```, allowing all the tokens to communicate.
Here is called a decoder attention block because it has triangular masking, and is usually used in autoregressive settings,like langauge modeling.

- Self-attention just means that keys and values are produced from the same source as queries. In cross-attention, the queries still get produced from x, but the keys and values from some other, external source(e.g. an encoder module)

- Scaled attention additional divides ```wei``` by 1/sqrt(head_size). This makes it so when input Q, K are unit variance, ```wei``` will be unit variance too and softmax will stay diffuse and not saturate too much. 

In [31]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) #* head_size**-0.5

In [32]:
k.var()

tensor(1.0700)

In [33]:
q.var()

tensor(0.9006)

In [34]:
wei.var() # equal to head_size

tensor(18.0429)

In [37]:
# if wei takes too positive or negative values, softmax will converge to a one hot vector.

torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [39]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # sharpen towards the max value
# here we are only aggregating information from a single node, so scaling is used to control the variance specially at initilization/

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])